# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_csv = "output_data/cities.csv"
weather_data_df = pd.read_csv(weather_data_csv)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data_df[['Lat', 'Lon']]
humidity = weather_data_df['Humidity']

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=400, point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Emerald,100,AU,1580365997,44,-23.53,148.17,93.20,14.99
1,Ancud,63,CL,1580365998,100,-41.87,-73.82,42.80,1.12
2,Castro,96,BR,1580366000,99,-24.79,-50.01,64.60,2.77
3,Jamestown,1,US,1580366001,92,42.10,-79.24,21.99,4.70
4,Bredasdorp,63,ZA,1580366003,72,-34.53,20.04,69.01,3.36


In [6]:
narrowed_city_df = weather_data_df[(weather_data_df['Max Temp'] > 75) & 
                                      (weather_data_df['Max Temp'] < 90) & 
                                      (weather_data_df['Wind Speed'] < 5) & 
                                      (weather_data_df['Humidity'] < 60)]
narrowed_city_df['Hotel Name'] = ""


C:\Users\henoc\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_city_df
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
35,Salalah,50,OM,1580366042,38,17.02,54.09,75.20,3.36,
42,Tura,16,IN,1580366050,39,25.52,90.22,79.07,3.40,
90,Guimbal,100,PH,1580366113,54,10.66,122.32,86.40,2.91,
160,Saint-Pierre,0,RE,1580366202,58,-21.34,55.48,84.20,4.70,
216,Nan,1,TH,1580366270,50,18.83,100.67,78.80,2.24,
293,Boende,62,CD,1580366367,51,-0.22,20.87,80.04,4.23,
317,Gopalur,0,IN,1580366397,42,16.62,77.80,84.69,4.92,
346,Myitkyina,51,MM,1580366433,31,25.38,97.40,75.07,2.37,
540,Ranibennur,0,IN,1580366669,35,14.62,75.62,84.00,4.12,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,
          "keyword": "Hotel",
          "types": "lodging",
          "key": g_key}

In [9]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    
    params['location'] = f'{lat},{lon}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\henoc\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
hotel_data = hotel_df[hotel_df['Hotel Name'] != '']
hotel_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
35,Salalah,50,OM,1580366042,38,17.02,54.09,75.20,3.36,Salalah Gardens Hotel
42,Tura,16,IN,1580366050,39,25.52,90.22,79.07,3.40,Hotel RIKMAN Continental
90,Guimbal,100,PH,1580366113,54,10.66,122.32,86.40,2.91,Shamrock Beach Resort
160,Saint-Pierre,0,RE,1580366202,58,-21.34,55.48,84.20,4.70,"Villa Delisle, Hotel & Spa"
346,Myitkyina,51,MM,1580366433,31,25.38,97.40,75.07,2.37,Cartel Hotel
540,Ranibennur,0,IN,1580366669,35,14.62,75.62,84.00,4.12,Hotel Mallikarjuna Residency


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
h_locations = hotel_data[["Lat", "Lon"]]

In [13]:
# Add marker layer ontop of heat map
symbol_layer = gmaps.symbol_layer(h_locations, scale=2, info_box_content=hotel_info)
marker_layer = gmaps.marker_layer(h_locations)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(symbol_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))